NOW WE TRAIN AND VALIDATE ON ALL (ITEM, SHOP) EVER SEEN IN DATA
Shold be:
train - all (item ,shop) previosly seen
test - dot product of (item, shop) in current dbn

Now it is as it should be

In [1]:
FIRST_N_MONTH_TO_DROP = 6

In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
from sklearn.preprocessing import TargetEncoder
import gc

import numba

import matplotlib.pyplot as plt


In [3]:
#!pip install torch

In [4]:
data_train = pd.read_csv('../data_cleaned/data_train.csv')
item_categories = pd.read_csv('../data_cleaned/item_categories.csv')
items = pd.read_csv('../data_cleaned/items.csv')
shops = pd.read_csv('../data_cleaned/shops.csv')
test = pd.read_csv('../data_cleaned/test.csv')
test['date_block_num'] = 34
data_train = data_train.merge(test, on=['item_id','shop_id','date_block_num'], how='outer').fillna(0)

In [5]:


data_train=data_train.reset_index()
data_train = data_train.sort_values(by='date_block_num')
train = data_train.copy()


In [6]:
def create_change(table):
    table=table.copy().fillna(0)
    cols = [col for col in table.columns if isinstance(col, int)]
    cols.sort()
    
    for col in cols[1:]:
        table[col] = ((table[col] / table[col-1]) - 1) * 100
        
    table = table.replace([np.inf], -1)
    table = table.replace([np.nan], -2)
    return table
        
    

In [7]:
import time
def create_item_lag_table(data,column='item_cnt_monthitem_id_lag_1', whole=None):#long - ~467s
    #whole contains lagged, which means for dbn = 24 lag_1 calculated on 23
    #(item cnt month for dbn==24 stored in lag_1 for dbn == 25 )
    #to make whole same format as data, we need to whole['date_block_num']-=1
    start = time.time()

    whole=whole.copy()
    new_table = data.copy()
    if 'lag' in column:
        whole['date_block_num'] -= 1
        whole = whole[whole['date_block_num'] >= 0]
        
    groped_item = whole.groupby(['item_id','date_block_num'])[column].unique()
    
    
    print(groped_item.map(lambda a:a[0]).describe())
    for item, dbn in groped_item.index:
        
        if len(groped_item[item, dbn]) != 1:
            print(item, dbn)
            print(groped_item[item, dbn])
            print('SMTH WENT WRONG')
        
        
        new_table.loc[new_table['item_id'] == item, dbn] = groped_item[item, dbn][0]

    end = time.time()
    print(f'time taken for column {column}: {end-start}')
    return new_table.fillna(0)
        

In [8]:
import time
def create_category_lag_table(data,column='item_cnt_monthitem_category_id_cat_lag_1', whole=None):#long - ~467s
    #whole contains lagged, which means for dbn = 24 lag_1 calculated on 23
    #(item cnt month for dbn==24 stored in lag_1 for dbn == 25 )
    #to make whole same format as data, we need to whole['date_block_num']-=1
    start = time.time()

    whole=whole.copy()
    new_table = data.copy()
    
    if 'lag' in column:
        whole['date_block_num'] -= 1
        whole = whole[whole['date_block_num'] >= 0]
    
    groped_item = whole.groupby(['item_category_id','date_block_num'])[column].unique()
    print(whole.date_block_num.unique())
    for item_category_id, dbn in groped_item.index:
        
        if len(groped_item[item_category_id, dbn]) != 1:
            print('SMTH WENT WRONG')
        
        
        new_table.loc[new_table['item_category_id'] == item_category_id, dbn] = groped_item[item_category_id, dbn][0]

    end = time.time()
    print(f'time taken for column {column}: {end-start}')
    return new_table.fillna(0)
        

In [9]:
import time
def create_item_city_lag_table(data,column=None, whole=None):#long - ~467s
    #whole contains lagged, which means for dbn = 24 lag_1 calculated on 23
    #(item cnt month for dbn==24 stored in lag_1 for dbn == 25 )
    #to make whole same format as data, we need to whole['date_block_num']-=1
    start = time.time()

    whole=whole.copy()
    new_table = data.copy()
    
    if 'lag' in column:
        whole['date_block_num'] -= 1
        whole = whole[whole['date_block_num'] >= 0]
    
    groped_item = whole.groupby(['item_id','date_block_num','city'])[column].unique()
    print(whole.date_block_num.unique())
    for item_id, dbn, city in groped_item.index:
        
        if len(groped_item[item_id, dbn, city]) != 1:
            print('SMTH WENT WRONG')
        
        
        new_table.loc[(new_table['item_id']==item_id)&(new_table['city']==city), \
        dbn] = groped_item[item_id, dbn, city][0]

    end = time.time()
    print(f'time taken for column {column}: {end-start}')
    return new_table.fillna(0)
        

In [10]:
import time
def create_category_shop_lag_table(data,column=None, whole=None):#long - ~467s
    #whole contains lagged, which means for dbn = 24 lag_1 calculated on 23
    #(item cnt month for dbn==24 stored in lag_1 for dbn == 25 )
    #to make whole same format as data, we need to whole['date_block_num']-=1
    start = time.time()

    whole=whole.copy()
    new_table = data.copy()
    
    if 'lag' in column:
        whole['date_block_num'] -= 1
        whole = whole[whole['date_block_num'] >= 0]
    
    groped_item = whole.groupby(['item_category_id','date_block_num','shop_id'])[column].unique()
    print(whole.date_block_num.unique())
    for item_category_id, dbn, shop_id in groped_item.index:
        
        if len(groped_item[item_category_id, dbn, shop_id]) != 1:
            print('SMTH WENT WRONG')
        
        
        new_table.loc[(new_table['item_category_id']==item_category_id)&(new_table['shop_id']==shop_id), \
        dbn] = groped_item[item_category_id, dbn, shop_id][0]

    end = time.time()
    print(f'time taken for column {column}: {end-start}')
    return new_table.fillna(0)
        

In [11]:
def create_item_shop_data(data, column='item_cnt_day'):
    if column == 'item_cnt_day':
        agg = 'sum'
    else:
        agg='mean'
    df = pd.pivot_table(data, 
                    index=['shop_id','item_id'],
                    columns=['date_block_num'],
                    values=[column],
                    aggfunc=agg, 
                    fill_value=0
                   ).reset_index()
   # print(df)
    train = pd.DataFrame({'shop_id':df['shop_id'], 'item_id':df['item_id']})
    
    item_cnt_day = df[column]
    train = pd.concat([train,item_cnt_day], axis=1)
    #print(items)
    merged = train.merge(items, how='left').merge(item_categories, how='left').merge(shops, how='left') \
        .drop(['item_name','item_category_name','shop_name'], axis=1)

    
    return merged

In [12]:
whole = pd.read_csv('../data.csv')

In [13]:
shop_city = whole.groupby('shop_id')['city'].unique().reset_index()

In [14]:
category_super_category = whole.groupby('item_category_id')['super_category'].unique().reset_index()

In [15]:
del whole
gc.collect()

0

In [16]:

merged = create_item_shop_data(data_train)

In [17]:
merged

,shop_id,item_id,0,1,2,3,4,5,6,7,...,26,27,28,29,30,31,32,33,34,item_category_id
0,2,27,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19
1,2,30,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
2,2,31,0.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,37
3,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,40
4,2,33,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521606,59,22162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,40
521607,59,22163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
521608,59,22164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,37
521609,59,22166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54


In [ ]:
"""
import gc
table_ema6_item = create_item_lag_table(merged,column='ema_6_item_cnt_month_item_id', whole=whole)
table_ema6_item.to_csv('data/ema_6_item_cnt_month_item_id.csv', index=False)
del table_ema6_item
gc.collect()

table_ema6_cat_shop = create_category_shop_lag_table(merged,column='ema_6_item_cnt_month_item_category_id_cat_shop_id', whole=whole)
table_ema6_cat_shop.to_csv('data/ema_6_item_cnt_month_item_category_id_cat_shop_id.csv', index=False)
del table_ema6_cat_shop
gc.collect()

table_item = create_item_lag_table(merged,column='date_block_num_diff', whole=whole)
table_item.to_csv('data/shop_item_date_block_num_diff.csv', index=False)
del table_item
gc.collect()


table_item_price = create_item_lag_table(merged,column='avg_item_priceitem_id_lag_1', whole=whole)
table_item_price.to_csv('data/item_price.csv', index=False)
"""
table_item_price = pd.read_csv('data/item_price.csv')
price_changes_items = create_change(table_item_price)
price_changes_items.to_csv('data/item_price_change.csv', index=False)
del table_item_price
del price_changes_items
gc.collect()

#prices = create_item_shop_data(data_train, column='item_price')
#prices.to_csv('data/shop_item_price.csv', index=False)

prices = pd.read_csv('data/shop_item_price.csv')
price_changes = create_change(prices)
price_changes.to_csv('data/shop_item_price_change.csv', index=False)
del price_changes
del prices
gc.collect()

cnt_changes_items_shops = create_change(merged)
cnt_changes_items_shops.to_csv('data/cnt_changes_items_shops.csv', index=False)
del cnt_changes_items_shops
gc.collect()

#table_item_cnt = create_item_lag_table(merged,column='item_cnt_monthitem_id_lag_1',  whole=whole)
#table_item_cnt.to_csv('data/item_cnt.csv', index=False)

table_item_cnt = pd.read_csv('data/item_cnt.csv')
cnt_changes_items = create_change(table_item_cnt)
cnt_changes_items.to_csv('data/cnt_changes_items.csv', index=False)
del cnt_changes_items
del table_item_cnt
gc.collect()
"""
table_category_cnt = create_category_lag_table(merged,
                                                column='item_cnt_monthitem_category_id_cat_lag_1', 
                                                whole=whole)
table_category_cnt.to_csv('data/category_cnt.csv', index=False)
del table_category_cnt
gc.collect()






table_item_shop_cnt = merged

table_ema6_cat_shop = od.read_csv('data/ema_6_item_cnt_month_item_category_id_cat_shop_id.csv')
table_ema6_item=pd.read_csv('data/ema_6_item_cnt_month_item_id.csv')

table_dbn_diff = pd.read_csv('data/shop_item_date_block_num_diff.csv')

table_item_price  = pd.read_csv ('data/item_price.csv')
table_item_price_changes = pd.read_csv('data/item_price_change.csv')

table_item_shop_price = pd.read_csv('data/shop_item_price.csv')
table_item_shop_price_changes  = pd.read_csv('data/shop_item_price_change.csv')

table_cnt_changes_items_shops=pd.read_csv('data/cnt_changes_items_shops.csv')

table_item_cnt = pd.read_csv('data/item_cnt.csv')
table_cnt_changes_items = pd.read_csv('data/cnt_changes_items.csv')

table_category_shop_cnt = pd.read_csv('data/shop_category_cnt.csv')





"""


In [18]:
merged=merged.merge(shop_city, on='shop_id', how='left')
merged=merged.merge(category_super_category, on='item_category_id', how='left')
merged['city'] = merged['city'].map(lambda a:a[0])
merged['super_category'] = merged['super_category'].map(lambda a:a[0])

In [19]:
names= [
       'ema_6_item_cnt_month_item_category_id_cat_shop_id',
       'ema_6_item_cnt_month_item_id',
       'shop_item_date_block_num_diff',
       'item_price',
       'shop_item_price',
       'item_price_change',
       'shop_item_price_change',
       'shop_category_cnt',
       'cnt_changes_items',
       'item_cnt',
       'cnt_changes_items_shops',]

#item_cnt
#shop_category_cnt
#

In [20]:
!pwd

/home/valeriy/python_projects/predict_future_sales/validation


In [21]:
len(names)

11

In [22]:
import pandas as pd

def rename_columns(df, name=None):
    
    new_columns = {}
    for col in df.columns:
        new_col = str(col)
        if new_col in ['shop_id','item_id','item_category_id','super_category','city']:
            continue

        else:
            #print(name)
            #print(new_col)
            new_col = name+'$'+new_col

        
        new_columns[col] = new_col
    
    #print(new_columns)
    df.rename(columns=new_columns, inplace=True)
    
    #return df


In [23]:
critical=['change','mean','ema']

def merge_csvs(df0, names):
    rename_columns(df0, 'shop_item_cnt')
    for df_name in names:
        df = pd.read_csv('data/'+df_name+'.csv')
        print(sum(df.memory_usage()/10**6))
        for i in range(FIRST_N_MONTH_TO_DROP):#exclude features where past can not be calculated
            
            if any(cr in df_name for cr in critical):
                
                df=df.drop(str(i), axis=1)
        
        rename_columns(df, df_name)
        
        df0 = pd.merge(df0, df, on=['shop_id','item_id','item_category_id'], how = 'left')
        del df
        gc.collect()

    return df0

In [24]:

merge_csvs(merged, names).to_csv('data/merged.csv', index=False)


158.56987600000002
158.56987600000002
158.56987600000002
158.56987600000002
158.56987600000002
158.56987600000002
158.56987600000002
158.56987600000002
158.56987600000002
158.56987600000002
158.56987600000002
